In [4]:
import numpy as np
import pandas as pd
from typing import List 
from sklearn import set_config
set_config(display='diagram')

In [10]:
import dill
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
data = pd.read_csv("https://factored-workshops.s3.amazonaws.com/taxi-trip-duration.csv")
# Limitar rango de datos
tiempo_minimo = 60 # 1 minuto
tiempo_maximo = 36000 # 10 horas
data = data[
    (data["trip_duration"] > tiempo_minimo) &
    (data["trip_duration"] < tiempo_maximo)
]

y = data["trip_duration"]
selected_columns = [
    'vendor_id',
    'pickup_datetime',
    'passenger_count',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'pickup_borough',
    'dropoff_borough'
]
input_df = data[selected_columns]
train_df, val_df, y_train, y_val = train_test_split(input_df, y, random_state=0)

In [12]:
with open("full_pipeline", "rb") as f: # Me falto poner el .pkl
    preprocessor = dill.load(f)

X_train = preprocessor.transform(train_df)
X_val = preprocessor.transform(val_df)

In [13]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score, mean_squared_log_error

def evaluar_predicciones(y_pred, y_true):
    mae = mean_absolute_error(y_pred=y_pred, y_true=y_true)
    mape = mean_absolute_percentage_error(y_pred=y_pred, y_true=y_true)
    rmse = mean_squared_error(y_pred=y_pred, y_true=y_true, squared=False)
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape}")
    print(f"RMSE: {rmse}")

In [14]:
from sklearn.dummy import DummyRegressor

dummy_model = DummyRegressor(strategy="mean")
dummy_model.fit(X_train, y_train)
y_train_dummy = dummy_model.predict(X_train)
y_val_dummy = dummy_model.predict(X_val)

print("TRAIN")
evaluar_predicciones(y_pred=y_train_dummy, y_true=y_train)

print("VALIDATION")
evaluar_predicciones(y_pred=y_val_dummy, y_true=y_val)

TRAIN
MAE: 468.61
MAPE: 0.95585624685749
RMSE: 683.1658889831872
VALIDATION
MAE: 468.42
MAPE: 0.9567650882594899
RMSE: 680.0030431781618


In [15]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
y_train_linear = lr.predict(X_train)
y_val_linear = lr.predict(X_val)

print("TRAIN")
evaluar_predicciones(y_pred=y_train_linear, y_true=y_train)

print("VALIDATION")
evaluar_predicciones(y_pred=y_val_linear, y_true=y_val)

TRAIN
MAE: 296.09
MAPE: 0.5533021393418195
RMSE: 487.05043254440756
VALIDATION
MAE: 295.55
MAPE: 0.5540681395532685
RMSE: 529.0958979710379


In [17]:
import mlflow
mlflow.sklearn.autolog()

In [18]:
with mlflow.start_run(run_name="dummy") as run:
    dummy_model.fit(X_train, y_train)
    y_pred_val = dummy_model.predict(X_val)
    val_mae = mean_absolute_error(y_pred=y_pred_val, y_true=y_val)
    val_rmse = mean_squared_error(y_pred=y_pred_val, y_true=y_val, squared=False)
    val_mape = mean_absolute_percentage_error(y_pred=y_pred_val, y_true=y_val)
    val_r2 = r2_score(y_pred=y_pred_val, y_true=y_val)

    mlflow.log_metric("val_mae", val_mae)
    mlflow.log_metric("val_rmse", val_rmse)
    mlflow.log_metric("val_mape", val_mape)
    mlflow.log_metric("val_r2", val_r2)

In [20]:
# mlflow ui se ejecuta en la terminal

In [22]:
with mlflow.start_run(run_name="linear_regression") as run:
    linear_model = LinearRegression()
    linear_model.fit(X_train, y_train)
    y_pred_val = linear_model.predict(X_val)
    val_mae = mean_absolute_error(y_pred=y_pred_val, y_true=y_val)
    val_rmse = mean_squared_error(y_pred=y_pred_val, y_true=y_val, squared=False)
    val_mape = mean_absolute_percentage_error(y_pred=y_pred_val, y_true=y_val)
    val_r2 = r2_score(y_pred=y_pred_val, y_true=y_val)

    mlflow.log_metric("val_mae", val_mae)
    mlflow.log_metric("val_rmse", val_rmse)
    mlflow.log_metric("val_mape", val_mape)
    mlflow.log_metric("val_r2", val_r2)
    mlflow.log_artifact("full_pipeline")

In [23]:
from sklearn.ensemble import RandomForestRegressor

with mlflow.start_run(run_name="random_forest") as run:
    rf_model = RandomForestRegressor(n_jobs=2)
    rf_model.fit(X_train, y_train)
    y_pred_val = rf_model.predict(X_val)
    val_mae = mean_absolute_error(y_pred=y_pred_val, y_true=y_val)
    val_rmse = mean_squared_error(y_pred=y_pred_val, y_true=y_val, squared=False)
    val_mape = mean_absolute_percentage_error(y_pred=y_pred_val, y_true=y_val)
    val_r2 = r2_score(y_pred=y_pred_val, y_true=y_val)

    mlflow.log_metric("val_mae", val_mae)
    mlflow.log_metric("val_rmse", val_rmse)
    mlflow.log_metric("val_mape", val_mape)
    mlflow.log_metric("val_r2", val_r2)
    mlflow.log_artifact("full_pipeline")

In [25]:
from lightgbm import LGBMRegressor

mlflow.lightgbm.autolog()
with mlflow.start_run(run_name="lgbm") as run:
    lgbm_model = LGBMRegressor()
    lgbm_model.fit(X_train, y_train)
    y_pred_val = lgbm_model.predict(X_val)
    log_metrics_mlflow(y_pred_val, y_val)
    mlflow.log_artifact("preprocesser.pkl")
    
    with open("lgbm_model.pkl", "wb") as f:
        dill.dump(lgbm_model, f)
    mlflow.log_artifact("lgbm_model.pkl")

OSError: dlopen(/Users/lorozcoceron/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/lorozcoceron/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found